In [2]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [38]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("09_db_cred.yaml")

In [36]:
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [86]:
# your code here 1

compare_charge_q = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = 'Chicago' 
         THEN 'YES'
         ELSE 'NO'
       END AS in_chicago
FROM   caseinit;
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)
compare_charge_d[["INCIDENT_CITY", "in_chicago"]]


,INCIDENT_CITY,in_chicago,in_chicago
0,,None,NO
1,,None,NO
2,,None,NO
3,,None,NO
4,,None,NO
...,...,...,...
272289,Chicago,None,YES
272290,Evergreen Park,None,NO
272291,Chicago,None,YES
272292,Chicago,None,YES


In [94]:
# your code here 2

compare_charge_q = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = 'Chicago'
         THEN 'YES'
         ELSE 'NO'
       END AS in_chicago
    FROM   caseinit
WHERE INCIDENT_CITY <> "";
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)
compare_charge_d[["INCIDENT_CITY", "in_chicago"]]


,INCIDENT_CITY,in_chicago,in_chicago
0,Oak Park,None,NO
1,Harvey,None,NO
2,Morton Grove,None,NO
3,Chicago,None,YES
4,Chicago,None,YES
...,...,...,...
251732,Chicago,None,YES
251733,Evergreen Park,None,NO
251734,Chicago,None,YES
251735,Chicago,None,YES


In [50]:
# your code here 3
# Use where to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted
#(is_in_diversion); pull the table with those counts


diff_c = pd.read_sql_query("""
SELECT is_in_diversion, COUNT(*) AS count_divert
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY is_in_diversion
ORDER BY count_divert DESC;
""", cnx)

diff_c

,is_in_diversion,count_divert
0,False,167171
1,True,6402


In [72]:
# your code here 4

prop_div = """
SELECT 
    SUM(CASE WHEN is_in_diversion = 'True' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS proportion_diverted 
    SELECT *,
           CASE 
               WHEN INCIDENT_CITY = 'Chicago' THEN 'YES' 
               ELSE 'NO' 
           END AS in_chicago2
    FROM caseinit
) AS tmp
WHERE in_chicago2 = 'YES';
"""

prop_div_d = pd.read_sql_query(prop_div, cnx)
prop_div_d


## so easy its like having a subtabele then just focus on selecting the proportion
#avg(is_div_book) as prop_div --> easier way to do it than the sum like the avg thing is built into SQL

,proportion_diverted
0,0.03688


In [74]:
# your code here 5


compare_prop_div = """
SELECT 
    in_chicago2 AS city_group,
    SUM(CASE WHEN is_in_diversion = 'True' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS proportion_diverted
FROM (
    SELECT *,
           CASE 
               WHEN INCIDENT_CITY = 'Chicago' THEN 'YES' 
               ELSE 'NO' 
           END AS in_chicago2
    FROM caseinit
) AS tmp
GROUP BY in_chicago2;
"""

compare_prop_div_d = pd.read_sql_query(compare_prop_div, cnx)
compare_prop_div_d




,city_group,proportion_diverted
0,NO,0.04091
1,YES,0.03688


In [ ]:
## HIS 5 SOLUTION
## step 5
query_propdiv_compare = """
SELECT 
    AVG(is_div_bool) as prop_div, 
    in_chicago_1
FROM  (
    SELECT 
        *,
        CASE 
            WHEN is_in_diversion = 'True' THEN 1
            ELSE 0
        END as is_div_bool,
        CASE 
            WHEN INCIDENT_CITY = 'Chicago' THEN 'Yes'
            ELSE 'No'
        END AS in_chicago_1
    FROM caseinit) AS d
WHERE INCIDENT_CITY <> ''
GROUP BY in_chicago_1
"""
prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)
prop_div_compare

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [ ]:
# your code here 1
## like having a dictionary from one to the other like you go through Case and when for all of htem to rename them

In [ ]:
# your code here 2